In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as FT

## Step 1 : Build Costum data class

In [ ]:
class BloodCellDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, csv_file, split, num_classes, num_bboxes):
        self.annotations = pd.read_csv(csv_file)
        self.dict_ = {"rbc":0, "wbc":1}
        self.annotations['label'] = self.annotations.label.map(self.dict_)
        self.img_dir = img_dir
        self.S = split
        self.C = num_classes
        self.B = num_bboxes
        
    def __len__(self):
        return len(os.listdir(self.img_dir))
    
    def __getitem__(self, index):
        ## image_name as parameter
        bboxes = []
        image_name = str(index)+".png"
        image_name_csv = "image-"+str(index)+".png"
        
        image = cv2.imread(os.path.join(self.img_dir,image_name))
        boxes = self.annotations[self.annotations["image"] == image_name_csv].drop(columns=["image"]).to_numpy()
        for box in boxes:
            xmin, ymin, xmax, ymax, class_label = float(box[1]), float(box[2]), float(box[3]), float(box[4]), box[5]
            x, y = ((xmin + xmax)/2)/image.shape[0], ((ymin + ymax)/2)/image.shape[1]
            width, height = abs(xmax - xmin)/image.shape[0], abs(ymax - ymin)/image.shape[1]
            bboxes.append([class_label, x,y,width,height]) 
        img = Image.open(os.path.join(self.img_dir,image_name))
        bboxes = torch.Tensor([x for x in bboxes])
        
        new_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
        img = new_transform(img)
        
         ## label_matrix
        label_matrix = torch.zeros((self.S, self.S, self.C + self.B * 5 ))
        
        for box in bboxes:
            class_label, x, y, width, height = box.tolist()
            i, j = int(self.S*y), int(self.S*x)
            x_cell, y_cell = self.S * x - j, self.S * y - i  
            width_cell, height_cell = (                                                                                                                
                width * self.S,
                height * self.S
            )
                
            if label_matrix[i, j, 2] == 0:
                label_matrix[i,j,2] = 1
                label_matrix[i,j,3:7] = torch.tensor([x_cell, y_cell, width_cell, height_cell])
                label_matrix[i,j,int(class_label)] = 1
        return img, label_matrix
    
    def __show_img_annotated__(self, index):
        img_file = str(index)+".png"
        img_file_csv =  "image-"+ str(index)+".png"
        image_path = os.path.join(self.img_dir,img_file)
        image_boxes = self.annotations[self.annotations["image"] == img_file_csv].drop(columns=["image"]).to_numpy()
        im = cv2.imread(image_path)
        for i in image_boxes:
            plt.plot([i[1],i[1],i[3],i[3],i[1]], [i[2],i[4],i[4],i[2],i[2]])
        plt.imshow(im)
        plt.show()
        new_transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((112, 112))])
        im = new_transform(im)

In [ ]:
blood_cell_dataset =  BloodCellDataset("dataset/images", "dataset/annotation.csv", 7, 2, 1)

In [ ]:
blood_cell_dataset.__show_img_annotated__(0)

## Step 3 : Utils Functions

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=1, C=2):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = S
        self.B = B
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)
        exists_box = target[..., 2].unsqueeze(3)
        
        # box coordinates loss
        box_predictions = exists_box * predictions[..., 3:7]
        box_targets = exists_box * target[..., 3:7]
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(torch.abs(box_predictions[..., 2:4] + 1e-6))
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])
        box_loss = self.mse(torch.flatten(box_predictions, end_dim=-2),torch.flatten(box_targets, end_dim=-2))
        
        # objects loss
        pred_box = predictions[..., 2:3]
        object_loss = self.mse(torch.flatten(exists_box * pred_box),torch.flatten(exists_box * target[..., 2:3]))
        
        # noobj loss
        no_object_loss = self.mse(torch.flatten((1 - exists_box) * predictions[..., 2:3], start_dim=1),torch.flatten((1 - exists_box) * target[..., 2:3], start_dim=1),)
        
        # class loss
        class_loss = self.mse(torch.flatten(exists_box * predictions[..., :2], end_dim=-2,),torch.flatten(exists_box * target[..., :2], end_dim=-2,),)
        
        # overall loss
        loss = (
            self.lambda_coord * box_loss 
            + object_loss  
            + self.lambda_noobj * no_object_loss 
            + class_loss 
        )
        return loss

## Build Network

In [ ]:
Model_architecture = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 512, 1, 1),
    "M",
    (3, 1024, 1, 1),
    (3, 1024, 1, 1)
]

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(Block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.Leakyrelu = nn.LeakyReLU(0.001)
    def forward(self, x):
        return self.Leakyrelu(self.batchnorm(self.conv(x)))

In [ ]:
class Net(nn.Module):
    def __init__(self, network_architecture, in_channels):
        super(Net, self).__init__()
        self.network_architecture = network_architecture
        self.in_channels = in_channels
        self.convLayers = self.convLayers_()
        
        self.fc1 = nn.Linear(7*7*1024, 1024)
        self.leakyrelu = nn.LeakyReLU(0.001)
        self.fc2 = nn.Linear(1024, 7*7*7)
        
    def convLayers_(self):
        layers = []
        for i in self.network_architecture:
            if type(i) == str:
                layers += [nn.MaxPool2d(2, stride=2)]
                
            elif type(i) == tuple:
                layers += [Block(self.in_channels, out_channels=i[1], kernel_size=i[0], stride=i[2], padding=i[3])]
                self.in_channels = i[1]
                
            elif type(i) == list:
                for _ in range(i[-1]):
                    layers += [Block(self.in_channels, out_channels=i[0][1], kernel_size=i[0][0], stride=i[0][2], padding=i[0][3])]
                    layers += [Block(in_channels=i[0][1], out_channels=i[1][1], kernel_size=i[1][0], stride=i[1][2], padding=i[1][3])]
                    self.in_channels = i[1][1]
        return nn.Sequential(*layers)

    
        
    def forward(self, x):
        x = self.convLayers(x)
        x = x.view(-1, 7*7*1024)
        x = self.fc1(x)
        x = self.leakyrelu(x)
        x = self.fc2(x) 
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
net = Net(network_architecture=Model_architecture, in_channels=3).to(device)

## Training

In [ ]:
IMG_SIZE = 224
LEARNING_RATE = 2e-5
BATCH_SIZE = 1 # 64 in original paper
EPOCHS = 100

In [ ]:
blood_cell_dataset =  BloodCellDataset("dataset/images", "dataset/annotation.csv", 7, 2, 1)
train_loader = DataLoader(
    dataset=blood_cell_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
loss_fn = YoloLoss()

In [ ]:
for epoch in range(EPOCHS):  
    print("[INFO] EPOCH ", epoch)
    loop = tqdm(train_loader, leave=True)
    mean_loss = []
    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        out = net(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        net.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")